# Portfolio Optimization Research Notebook

**Project:** Finance Software Suite  
**Module:** Portfolio Optimization  
**Purpose:** Compare Mean-Variance, Min-Variance, Equal-Weight, and Risk-Parity strategies  


---

**Table of Contents**

1. [Setup & Data](#1-setup--data)
2. [Returns Analysis](#2-returns-analysis)
3. [Mean-Variance Optimization (Markowitz)](#3-mean-variance-optimization-markowitz)
4. [Risk Metrics](#4-risk-metrics)
5. [Equal Weight Portfolio (Baseline)](#5-equal-weight-portfolio-baseline)
6. [Risk Parity](#6-risk-parity)
7. [Strategy Comparison](#7-strategy-comparison)
8. [Next Steps / Extension Points](#8-next-steps--extension-points)

---
## 1. Setup & Data

Install dependencies (safe to re-run on Colab) and generate synthetic price data
that matches the project's `DummyDataProvider`.

In [ ]:
# --- Colab / pip install cell ---
# Run this cell first on Google Colab. It is safe to re-run locally.
import subprocess, sys

_REQUIRED = [
    "numpy",
    "pandas",
    "scipy",
    "plotly",
    "matplotlib",
]

for pkg in _REQUIRED:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("All dependencies available.")

In [ ]:
# --- Optional: add project src/ to path so we can import local modules ---
# This block is a no-op on Colab (the path simply won't exist).
import os, sys, pathlib

_PROJECT_ROOT = pathlib.Path(os.getcwd()).parent  # notebooks/ -> project root
if (_PROJECT_ROOT / "src").exists():
    sys.path.insert(0, str(_PROJECT_ROOT))
    print(f"Project root added to sys.path: {_PROJECT_ROOT}")
else:
    print("Running standalone (project src/ not found). All logic is inlined below.")

In [ ]:
# --- Core imports ---
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import stats as sp_stats

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Reproducibility
np.random.seed(42)

# Display settings
pd.set_option("display.float_format", "{:.4f}".format)
pd.set_option("display.max_columns", 20)

# Constants used throughout the notebook
RISK_FREE_RATE = 0.02        # 2 % annual risk-free rate
TRADING_DAYS   = 252         # annualization factor
NUM_PORTFOLIOS = 10_000      # Monte Carlo simulations

print(f"numpy {np.__version__}, pandas {pd.__version__}")
print(f"Risk-free rate: {RISK_FREE_RATE:.0%}, Trading days: {TRADING_DAYS}")

In [ ]:
# --- Generate synthetic price data ---
# Mirrors src/data/dummy_provider.py -> DummyDataProvider.get_portfolio_data()

np.random.seed(42)  # reset seed for reproducible data

dates   = pd.bdate_range(start="2022-01-03", end="2023-12-29")
tickers = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]

# Annualized parameters per asset
annual_returns = [0.12, 0.10, 0.14, 0.08, 0.20]
annual_vols    = [0.25, 0.22, 0.20, 0.28, 0.45]

prices = pd.DataFrame(index=dates, columns=tickers, dtype=float)
for i, ticker in enumerate(tickers):
    daily_ret = annual_returns[i] / TRADING_DAYS
    daily_vol = annual_vols[i] / np.sqrt(TRADING_DAYS)
    log_returns = np.random.normal(daily_ret, daily_vol, len(dates))
    prices[ticker] = 100.0 * np.cumprod(1 + log_returns)

n_days   = len(prices)
n_assets = len(tickers)

print(f"Generated {n_days} business days of price data for {n_assets} assets")
print(f"Date range: {prices.index[0].date()} to {prices.index[-1].date()}")
prices.tail()

In [ ]:
# --- Price chart ---
fig = go.Figure()
for col in prices.columns:
    fig.add_trace(go.Scatter(
        x=prices.index, y=prices[col],
        mode="lines", name=col,
    ))

fig.update_layout(
    title="Synthetic Asset Prices (Base = 100)",
    xaxis_title="Date",
    yaxis_title="Price ($)",
    height=500,
    template="plotly_white",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

---
## 2. Returns Analysis

Compute daily log returns and explore their statistical properties.

In [ ]:
# --- Daily simple returns ---
daily_returns = prices.pct_change().dropna()

print(f"Daily returns shape: {daily_returns.shape}")
daily_returns.head()

In [ ]:
# --- Return distribution histograms (matplotlib) ---
fig, axes = plt.subplots(1, n_assets, figsize=(18, 4), sharey=True)

for ax, ticker in zip(axes, tickers):
    data = daily_returns[ticker]
    ax.hist(data, bins=50, edgecolor="white", alpha=0.8, color="steelblue")
    ax.axvline(data.mean(), color="red", linewidth=1.2, linestyle="--", label="Mean")
    ax.set_title(ticker, fontsize=12, fontweight="bold")
    ax.set_xlabel("Daily Return")
    ax.xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
    ax.legend(fontsize=8)

axes[0].set_ylabel("Frequency")
fig.suptitle("Daily Return Distributions", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# --- Summary statistics table ---
summary = pd.DataFrame({
    "Ann. Mean Return": daily_returns.mean() * TRADING_DAYS,
    "Ann. Volatility":  daily_returns.std() * np.sqrt(TRADING_DAYS),
    "Skewness":         daily_returns.skew(),
    "Excess Kurtosis":  daily_returns.kurtosis(),
    "Min Daily":        daily_returns.min(),
    "Max Daily":        daily_returns.max(),
})

# Format for display
summary_display = summary.copy()
summary_display["Ann. Mean Return"]  = summary_display["Ann. Mean Return"].apply(lambda x: f"{x:.2%}")
summary_display["Ann. Volatility"]   = summary_display["Ann. Volatility"].apply(lambda x: f"{x:.2%}")
summary_display["Min Daily"]         = summary_display["Min Daily"].apply(lambda x: f"{x:.2%}")
summary_display["Max Daily"]         = summary_display["Max Daily"].apply(lambda x: f"{x:.2%}")

print("Return Summary Statistics")
print("=" * 80)
summary_display

In [ ]:
# --- Correlation matrix heatmap ---
corr_matrix = daily_returns.corr()

fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    aspect="auto",
    title="Asset Return Correlation Matrix",
)
fig.update_layout(height=450, template="plotly_white")
fig.show()

---
## 3. Mean-Variance Optimization (Markowitz)

The classical Markowitz framework finds portfolio weights that maximize
risk-adjusted return (Sharpe ratio) or minimize variance.

**Steps:**
1. Estimate expected returns and the covariance matrix from historical data.
2. Run a Monte Carlo simulation of 10,000 random portfolios to visualize the opportunity set.
3. Use `scipy.optimize.minimize` to find the max-Sharpe and min-variance portfolios.

In [ ]:
# --- Expected returns and covariance ---
mean_returns = daily_returns.mean() * TRADING_DAYS
cov_matrix   = daily_returns.cov()  * TRADING_DAYS

print("Annualized Expected Returns")
for t, r in mean_returns.items():
    print(f"  {t}: {r:>8.2%}")

print("\nAnnualized Covariance Matrix")
cov_matrix

In [ ]:
# --- Monte Carlo simulation: 10,000 random portfolios ---
np.random.seed(42)

mc_returns   = np.zeros(NUM_PORTFOLIOS)
mc_vols      = np.zeros(NUM_PORTFOLIOS)
mc_sharpes   = np.zeros(NUM_PORTFOLIOS)
mc_weights   = np.zeros((NUM_PORTFOLIOS, n_assets))

cov_values = cov_matrix.values

for i in range(NUM_PORTFOLIOS):
    w = np.random.dirichlet(np.ones(n_assets))
    mc_weights[i] = w
    ret = np.dot(w, mean_returns.values)
    vol = np.sqrt(np.dot(w.T, np.dot(cov_values, w)))
    mc_returns[i] = ret
    mc_vols[i]    = vol
    mc_sharpes[i] = (ret - RISK_FREE_RATE) / vol if vol > 0 else 0.0

print(f"Simulated {NUM_PORTFOLIOS:,} random portfolios")
print(f"Sharpe range: [{mc_sharpes.min():.2f}, {mc_sharpes.max():.2f}]")

In [ ]:
# --- Optimize for Maximum Sharpe Ratio ---
def neg_sharpe(w, mean_ret, cov_mat, rf):
    """Negative Sharpe ratio (minimize to find max Sharpe)."""
    port_ret = np.dot(w, mean_ret)
    port_vol = np.sqrt(np.dot(w.T, np.dot(cov_mat, w)))
    return -(port_ret - rf) / port_vol if port_vol > 0 else 0

constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]
bounds      = tuple((0.0, 1.0) for _ in range(n_assets))
w0          = np.array([1.0 / n_assets] * n_assets)

opt_sharpe_result = minimize(
    neg_sharpe, w0, args=(mean_returns.values, cov_values, RISK_FREE_RATE),
    method="SLSQP", bounds=bounds, constraints=constraints,
)

w_max_sharpe   = opt_sharpe_result.x
ret_max_sharpe = float(np.dot(w_max_sharpe, mean_returns.values))
vol_max_sharpe = float(np.sqrt(np.dot(w_max_sharpe.T, np.dot(cov_values, w_max_sharpe))))
sr_max_sharpe  = (ret_max_sharpe - RISK_FREE_RATE) / vol_max_sharpe

print("=== Max Sharpe Portfolio ===")
print(f"  Expected Return: {ret_max_sharpe:.2%}")
print(f"  Volatility:      {vol_max_sharpe:.2%}")
print(f"  Sharpe Ratio:    {sr_max_sharpe:.4f}")
print(f"  Weights:")
for t, wt in zip(tickers, w_max_sharpe):
    print(f"    {t}: {wt:.4f} ({wt:.1%})")

In [ ]:
# --- Optimize for Minimum Variance ---
def portfolio_variance(w, cov_mat):
    """Portfolio variance."""
    return np.dot(w.T, np.dot(cov_mat, w))

opt_minvar_result = minimize(
    portfolio_variance, w0, args=(cov_values,),
    method="SLSQP", bounds=bounds, constraints=constraints,
)

w_min_var   = opt_minvar_result.x
ret_min_var = float(np.dot(w_min_var, mean_returns.values))
vol_min_var = float(np.sqrt(np.dot(w_min_var.T, np.dot(cov_values, w_min_var))))
sr_min_var  = (ret_min_var - RISK_FREE_RATE) / vol_min_var

print("=== Minimum Variance Portfolio ===")
print(f"  Expected Return: {ret_min_var:.2%}")
print(f"  Volatility:      {vol_min_var:.2%}")
print(f"  Sharpe Ratio:    {sr_min_var:.4f}")
print(f"  Weights:")
for t, wt in zip(tickers, w_min_var):
    print(f"    {t}: {wt:.4f} ({wt:.1%})")

In [ ]:
# --- Efficient Frontier scatter plot ---
fig = go.Figure()

# Simulated portfolios (colored by Sharpe)
fig.add_trace(go.Scatter(
    x=mc_vols, y=mc_returns, mode="markers",
    marker=dict(
        color=mc_sharpes, colorscale="Viridis", showscale=True,
        colorbar=dict(title="Sharpe Ratio"),
        size=3, opacity=0.5,
    ),
    text=[f"Sharpe: {s:.2f}" for s in mc_sharpes],
    name="Simulated (n=10,000)",
))

# Max Sharpe
fig.add_trace(go.Scatter(
    x=[vol_max_sharpe], y=[ret_max_sharpe], mode="markers",
    marker=dict(color="red", size=16, symbol="star", line=dict(width=1, color="black")),
    name=f"Max Sharpe ({sr_max_sharpe:.2f})",
))

# Min Variance
fig.add_trace(go.Scatter(
    x=[vol_min_var], y=[ret_min_var], mode="markers",
    marker=dict(color="blue", size=14, symbol="diamond", line=dict(width=1, color="black")),
    name=f"Min Variance ({sr_min_var:.2f})",
))

# Individual assets
individual_vols = [float(np.sqrt(cov_matrix.loc[t, t])) for t in tickers]
individual_rets = [float(mean_returns[t]) for t in tickers]
for t, iv, ir in zip(tickers, individual_vols, individual_rets):
    fig.add_trace(go.Scatter(
        x=[iv], y=[ir], mode="markers+text",
        marker=dict(color="orange", size=10, symbol="circle"),
        text=[t], textposition="top center",
        name=t, showlegend=False,
    ))

fig.update_layout(
    title="Efficient Frontier (Monte Carlo + Optimized Portfolios)",
    xaxis_title="Annualized Volatility",
    yaxis_title="Annualized Expected Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    height=600,
    template="plotly_white",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [ ]:
# --- Optimal weights bar chart ---
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Max Sharpe Weights", "Min Variance Weights"),
)

fig.add_trace(go.Bar(
    x=tickers, y=w_max_sharpe,
    text=[f"{w:.1%}" for w in w_max_sharpe],
    textposition="auto", marker_color="indianred",
    name="Max Sharpe",
), row=1, col=1)

fig.add_trace(go.Bar(
    x=tickers, y=w_min_var,
    text=[f"{w:.1%}" for w in w_min_var],
    textposition="auto", marker_color="steelblue",
    name="Min Variance",
), row=1, col=2)

fig.update_yaxes(tickformat=".0%", title_text="Weight", row=1, col=1)
fig.update_yaxes(tickformat=".0%", row=1, col=2)
fig.update_layout(height=400, template="plotly_white", showlegend=False,
                  title_text="Portfolio Weight Comparison")
fig.show()

---
## 4. Risk Metrics

Compute comprehensive risk metrics for the **Max Sharpe** portfolio:
- Value at Risk (parametric and historical)
- Conditional VaR / Expected Shortfall
- Maximum Drawdown
- Sortino Ratio
- Calmar Ratio
- Rolling volatility

In [ ]:
# --- Portfolio daily returns (max Sharpe weights) ---
port_daily_returns = daily_returns.dot(w_max_sharpe)
port_daily_returns.name = "Max Sharpe Portfolio"

ann_port_ret = port_daily_returns.mean() * TRADING_DAYS
ann_port_vol = port_daily_returns.std()  * np.sqrt(TRADING_DAYS)

print(f"Portfolio daily returns: {len(port_daily_returns)} observations")
print(f"Annualized return: {ann_port_ret:.2%}")
print(f"Annualized vol:    {ann_port_vol:.2%}")

In [ ]:
# --- Value at Risk ---

# Parametric VaR (assumes normal distribution)
mu_daily  = port_daily_returns.mean()
std_daily = port_daily_returns.std()

var_95_parametric = mu_daily + sp_stats.norm.ppf(0.05) * std_daily
var_99_parametric = mu_daily + sp_stats.norm.ppf(0.01) * std_daily

# Historical VaR (empirical quantile)
var_95_historical = float(np.percentile(port_daily_returns, 5))
var_99_historical = float(np.percentile(port_daily_returns, 1))

print("=== Value at Risk (daily) ===")
print(f"  Parametric VaR 95%: {var_95_parametric:.4%}")
print(f"  Parametric VaR 99%: {var_99_parametric:.4%}")
print(f"  Historical VaR 95%: {var_95_historical:.4%}")
print(f"  Historical VaR 99%: {var_99_historical:.4%}")

In [ ]:
# --- Conditional VaR (Expected Shortfall) ---
# Average loss beyond VaR threshold

cvar_95 = float(port_daily_returns[port_daily_returns <= var_95_historical].mean())
cvar_99 = float(port_daily_returns[port_daily_returns <= var_99_historical].mean())

print("=== Conditional VaR / Expected Shortfall (daily) ===")
print(f"  CVaR 95%: {cvar_95:.4%}")
print(f"  CVaR 99%: {cvar_99:.4%}")
print(f"")
print(f"  Interpretation: On the worst 5% of days, the portfolio loses")
print(f"  {abs(cvar_95):.2%} on average.")

In [ ]:
# --- Maximum Drawdown ---
cumulative     = (1 + port_daily_returns).cumprod()
rolling_max    = cumulative.cummax()
drawdown       = (cumulative - rolling_max) / rolling_max
max_drawdown   = float(drawdown.min())
max_dd_date    = drawdown.idxmin()

print(f"Maximum Drawdown: {max_drawdown:.2%}")
print(f"Occurred on:      {max_dd_date.date()}")

# Drawdown plot
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    subplot_titles=("Cumulative Return", "Drawdown"),
    vertical_spacing=0.08,
)

fig.add_trace(go.Scatter(
    x=cumulative.index, y=cumulative.values,
    mode="lines", name="Cumulative Return",
    line=dict(color="steelblue"),
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=drawdown.index, y=drawdown.values,
    mode="lines", name="Drawdown",
    fill="tozeroy", line=dict(color="indianred"),
), row=2, col=1)

fig.add_hline(y=max_drawdown, line_dash="dash", line_color="black",
              annotation_text=f"Max DD: {max_drawdown:.1%}", row=2, col=1)

fig.update_yaxes(tickformat=".0%", row=2, col=1)
fig.update_layout(height=600, template="plotly_white",
                  title_text="Max Sharpe Portfolio: Cumulative Return & Drawdown")
fig.show()

In [ ]:
# --- Sortino Ratio ---
# Uses downside deviation (only negative returns) instead of total std

downside_returns = port_daily_returns[port_daily_returns < 0]
downside_std     = float(downside_returns.std() * np.sqrt(TRADING_DAYS))
sortino_ratio    = (ann_port_ret - RISK_FREE_RATE) / downside_std if downside_std > 0 else 0

print(f"Sortino Ratio: {sortino_ratio:.4f}")
print(f"  (Sharpe uses total vol = {ann_port_vol:.2%}, "
      f"Sortino uses downside vol = {downside_std:.2%})")

In [ ]:
# --- Calmar Ratio ---
# Annualized return / |max drawdown|

calmar_ratio = ann_port_ret / abs(max_drawdown) if max_drawdown != 0 else 0

print(f"Calmar Ratio: {calmar_ratio:.4f}")
print(f"  (Ann. Return = {ann_port_ret:.2%}, Max DD = {max_drawdown:.2%})")

In [ ]:
# --- Rolling volatility (30-day window) ---
rolling_vol = port_daily_returns.rolling(window=30).std() * np.sqrt(TRADING_DAYS)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=rolling_vol.index, y=rolling_vol.values,
    mode="lines", name="30-Day Rolling Vol",
    line=dict(color="steelblue"),
))
fig.add_hline(y=ann_port_vol, line_dash="dash", line_color="red",
              annotation_text=f"Full-period vol: {ann_port_vol:.1%}")

fig.update_layout(
    title="Max Sharpe Portfolio: 30-Day Rolling Annualized Volatility",
    xaxis_title="Date",
    yaxis_title="Annualized Volatility",
    yaxis_tickformat=".0%",
    height=400,
    template="plotly_white",
)
fig.show()

In [ ]:
# --- Risk metrics summary table ---
risk_summary = pd.DataFrame({
    "Metric": [
        "Parametric VaR 95% (daily)",
        "Parametric VaR 99% (daily)",
        "Historical VaR 95% (daily)",
        "Historical VaR 99% (daily)",
        "CVaR / Expected Shortfall 95%",
        "CVaR / Expected Shortfall 99%",
        "Maximum Drawdown",
        "Sharpe Ratio",
        "Sortino Ratio",
        "Calmar Ratio",
        "Skewness",
        "Excess Kurtosis",
    ],
    "Value": [
        f"{var_95_parametric:.4%}",
        f"{var_99_parametric:.4%}",
        f"{var_95_historical:.4%}",
        f"{var_99_historical:.4%}",
        f"{cvar_95:.4%}",
        f"{cvar_99:.4%}",
        f"{max_drawdown:.2%}",
        f"{sr_max_sharpe:.4f}",
        f"{sortino_ratio:.4f}",
        f"{calmar_ratio:.4f}",
        f"{port_daily_returns.skew():.4f}",
        f"{port_daily_returns.kurtosis():.4f}",
    ],
})
risk_summary.set_index("Metric", inplace=True)
risk_summary

---
## 5. Equal Weight Portfolio (Baseline)

A naive 1/N allocation serves as a strong baseline.  
Research (DeMiguel, Garlappi, Uppal, 2009) shows that equal-weight often
outperforms optimized portfolios out-of-sample due to estimation error in
expected returns.

In [ ]:
# --- Equal weight portfolio ---
w_equal = np.array([1.0 / n_assets] * n_assets)

ret_equal = float(np.dot(w_equal, mean_returns.values))
vol_equal = float(np.sqrt(np.dot(w_equal.T, np.dot(cov_values, w_equal))))
sr_equal  = (ret_equal - RISK_FREE_RATE) / vol_equal

# Daily returns for this portfolio
port_equal_returns = daily_returns.dot(w_equal)

# Downside metrics
ds_equal     = port_equal_returns[port_equal_returns < 0]
ds_std_equal = float(ds_equal.std() * np.sqrt(TRADING_DAYS)) if len(ds_equal) > 0 else 0
sortino_equal = (ret_equal - RISK_FREE_RATE) / ds_std_equal if ds_std_equal > 0 else 0

# Max drawdown
cum_equal    = (1 + port_equal_returns).cumprod()
dd_equal     = (cum_equal - cum_equal.cummax()) / cum_equal.cummax()
max_dd_equal = float(dd_equal.min())
calmar_equal = ret_equal / abs(max_dd_equal) if max_dd_equal != 0 else 0

print("=== Equal Weight (1/N) Portfolio ===")
print(f"  Weights:         {dict(zip(tickers, w_equal.round(4)))}")
print(f"  Expected Return: {ret_equal:.2%}")
print(f"  Volatility:      {vol_equal:.2%}")
print(f"  Sharpe Ratio:    {sr_equal:.4f}")
print(f"  Sortino Ratio:   {sortino_equal:.4f}")
print(f"  Max Drawdown:    {max_dd_equal:.2%}")
print(f"  Calmar Ratio:    {calmar_equal:.4f}")
print()
print("--- vs Max Sharpe ---")
print(f"  Return diff:  {ret_max_sharpe - ret_equal:+.2%}")
print(f"  Vol diff:     {vol_max_sharpe - vol_equal:+.2%}")
print(f"  Sharpe diff:  {sr_max_sharpe - sr_equal:+.4f}")

---
## 6. Risk Parity

### Concept

**Risk Parity** allocates weights so that each asset contributes *equally*
to the total portfolio risk (volatility).  Unlike mean-variance optimization,
it does **not** use expected returns as inputs, making it more robust to
estimation error.

**Risk contribution** of asset $i$:

$$RC_i = w_i \cdot \frac{(\Sigma w)_i}{\sqrt{w^T \Sigma w}}$$

The optimization objective is:

$$\min_w \sum_{i=1}^{N} \left( RC_i - \frac{\sigma_p}{N} \right)^2$$

subject to $\sum w_i = 1$ and $w_i \geq 0$.

In [ ]:
# --- Risk Parity implementation ---

def risk_contribution(w, cov_mat):
    """Compute the risk contribution of each asset."""
    port_vol = np.sqrt(np.dot(w.T, np.dot(cov_mat, w)))
    # Marginal risk contribution
    marginal = np.dot(cov_mat, w) / port_vol
    # Risk contribution = weight * marginal contribution
    rc = w * marginal
    return rc


def risk_parity_objective(w, cov_mat):
    """
    Objective: minimize the sum of squared differences between
    each asset's risk contribution and the target (equal) contribution.
    """
    rc = risk_contribution(w, cov_mat)
    target = np.sum(rc) / len(w)  # equal risk contribution target
    return np.sum((rc - target) ** 2)


# Optimize
rp_constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]
rp_bounds      = tuple((0.01, 1.0) for _ in range(n_assets))  # min 1% to avoid zeros
rp_x0          = np.array([1.0 / n_assets] * n_assets)

rp_result = minimize(
    risk_parity_objective, rp_x0, args=(cov_values,),
    method="SLSQP", bounds=rp_bounds, constraints=rp_constraints,
    options={"ftol": 1e-12, "maxiter": 1000},
)

w_risk_parity = rp_result.x
ret_rp = float(np.dot(w_risk_parity, mean_returns.values))
vol_rp = float(np.sqrt(np.dot(w_risk_parity.T, np.dot(cov_values, w_risk_parity))))
sr_rp  = (ret_rp - RISK_FREE_RATE) / vol_rp

# Verify equal risk contributions
rc = risk_contribution(w_risk_parity, cov_values)
rc_pct = rc / rc.sum() * 100

print("=== Risk Parity Portfolio ===")
print(f"  Optimization converged: {rp_result.success}")
print(f"  Expected Return: {ret_rp:.2%}")
print(f"  Volatility:      {vol_rp:.2%}")
print(f"  Sharpe Ratio:    {sr_rp:.4f}")
print(f"  Weights and Risk Contributions:")
for t, wt, rci in zip(tickers, w_risk_parity, rc_pct):
    print(f"    {t}: weight={wt:.4f} ({wt:.1%}), risk contrib={rci:.1f}%")

In [ ]:
# --- Risk contribution comparison: Risk Parity vs Mean-Variance ---
rc_mv = risk_contribution(w_max_sharpe, cov_values)
rc_mv_pct = rc_mv / rc_mv.sum() * 100

rc_rp_pct = rc / rc.sum() * 100

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Max Sharpe: Risk Contributions", "Risk Parity: Risk Contributions"),
)

fig.add_trace(go.Bar(
    x=tickers, y=rc_mv_pct,
    text=[f"{v:.1f}%" for v in rc_mv_pct],
    textposition="auto", marker_color="indianred",
    name="Max Sharpe",
), row=1, col=1)

fig.add_trace(go.Bar(
    x=tickers, y=rc_rp_pct,
    text=[f"{v:.1f}%" for v in rc_rp_pct],
    textposition="auto", marker_color="seagreen",
    name="Risk Parity",
), row=1, col=2)

# Add target line (20% each)
for col in [1, 2]:
    fig.add_hline(y=20, line_dash="dash", line_color="black",
                  annotation_text="Target: 20%", row=1, col=col)

fig.update_yaxes(title_text="Risk Contribution (%)", range=[0, max(rc_mv_pct.max(), 40)], row=1, col=1)
fig.update_yaxes(range=[0, max(rc_mv_pct.max(), 40)], row=1, col=2)
fig.update_layout(height=400, template="plotly_white", showlegend=False,
                  title_text="Risk Contribution Comparison")
fig.show()

In [ ]:
# --- Compute risk-parity drawdown and Sortino for later comparison ---
port_rp_returns = daily_returns.dot(w_risk_parity)
ds_rp     = port_rp_returns[port_rp_returns < 0]
ds_std_rp = float(ds_rp.std() * np.sqrt(TRADING_DAYS)) if len(ds_rp) > 0 else 0
sortino_rp = (ret_rp - RISK_FREE_RATE) / ds_std_rp if ds_std_rp > 0 else 0

cum_rp     = (1 + port_rp_returns).cumprod()
dd_rp      = (cum_rp - cum_rp.cummax()) / cum_rp.cummax()
max_dd_rp  = float(dd_rp.min())
calmar_rp  = ret_rp / abs(max_dd_rp) if max_dd_rp != 0 else 0

print(f"Risk Parity - Sortino: {sortino_rp:.4f}, Max DD: {max_dd_rp:.2%}, Calmar: {calmar_rp:.4f}")

---
## 7. Strategy Comparison

Compare all four strategies head-to-head.

In [ ]:
# --- Compute min-variance drawdown and Sortino ---
port_mv_returns = daily_returns.dot(w_min_var)
ds_mv     = port_mv_returns[port_mv_returns < 0]
ds_std_mv = float(ds_mv.std() * np.sqrt(TRADING_DAYS)) if len(ds_mv) > 0 else 0
sortino_mv = (ret_min_var - RISK_FREE_RATE) / ds_std_mv if ds_std_mv > 0 else 0

cum_mv     = (1 + port_mv_returns).cumprod()
dd_mv      = (cum_mv - cum_mv.cummax()) / cum_mv.cummax()
max_dd_mv  = float(dd_mv.min())
calmar_mv  = ret_min_var / abs(max_dd_mv) if max_dd_mv != 0 else 0

print(f"Min Variance - Sortino: {sortino_mv:.4f}, Max DD: {max_dd_mv:.2%}, Calmar: {calmar_mv:.4f}")

In [ ]:
# --- Side-by-side comparison table ---
comparison = pd.DataFrame({
    "Max Sharpe": {
        "Ann. Return":    f"{ret_max_sharpe:.2%}",
        "Ann. Volatility": f"{vol_max_sharpe:.2%}",
        "Sharpe Ratio":   f"{sr_max_sharpe:.4f}",
        "Sortino Ratio":  f"{sortino_ratio:.4f}",
        "Max Drawdown":   f"{max_drawdown:.2%}",
        "Calmar Ratio":   f"{calmar_ratio:.4f}",
    },
    "Min Variance": {
        "Ann. Return":    f"{ret_min_var:.2%}",
        "Ann. Volatility": f"{vol_min_var:.2%}",
        "Sharpe Ratio":   f"{sr_min_var:.4f}",
        "Sortino Ratio":  f"{sortino_mv:.4f}",
        "Max Drawdown":   f"{max_dd_mv:.2%}",
        "Calmar Ratio":   f"{calmar_mv:.4f}",
    },
    "Equal Weight": {
        "Ann. Return":    f"{ret_equal:.2%}",
        "Ann. Volatility": f"{vol_equal:.2%}",
        "Sharpe Ratio":   f"{sr_equal:.4f}",
        "Sortino Ratio":  f"{sortino_equal:.4f}",
        "Max Drawdown":   f"{max_dd_equal:.2%}",
        "Calmar Ratio":   f"{calmar_equal:.4f}",
    },
    "Risk Parity": {
        "Ann. Return":    f"{ret_rp:.2%}",
        "Ann. Volatility": f"{vol_rp:.2%}",
        "Sharpe Ratio":   f"{sr_rp:.4f}",
        "Sortino Ratio":  f"{sortino_rp:.4f}",
        "Max Drawdown":   f"{max_dd_rp:.2%}",
        "Calmar Ratio":   f"{calmar_rp:.4f}",
    },
})

print("Strategy Comparison")
print("=" * 80)
comparison

In [ ]:
# --- Weight comparison across strategies ---
weight_df = pd.DataFrame({
    "Max Sharpe":  w_max_sharpe,
    "Min Variance": w_min_var,
    "Equal Weight": w_equal,
    "Risk Parity":  w_risk_parity,
}, index=tickers)

print("Portfolio Weights")
print("=" * 60)
weight_display = weight_df.copy()
for col in weight_display.columns:
    weight_display[col] = weight_display[col].apply(lambda x: f"{x:.1%}")
weight_display

In [ ]:
# --- Bar chart: Return, Volatility, Sharpe across strategies ---
strategies = ["Max Sharpe", "Min Variance", "Equal Weight", "Risk Parity"]
returns_list = [ret_max_sharpe, ret_min_var, ret_equal, ret_rp]
vols_list    = [vol_max_sharpe, vol_min_var, vol_equal, vol_rp]
sharpe_list  = [sr_max_sharpe, sr_min_var, sr_equal, sr_rp]
colors       = ["indianred", "steelblue", "goldenrod", "seagreen"]

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Annualized Return", "Annualized Volatility", "Sharpe Ratio"),
)

for i, (strat, ret, vol, sr, color) in enumerate(
    zip(strategies, returns_list, vols_list, sharpe_list, colors)
):
    fig.add_trace(go.Bar(
        x=[strat], y=[ret], name=strat,
        marker_color=color, text=[f"{ret:.1%}"], textposition="auto",
        showlegend=(i == 0),  # only show legend once per strategy name
        legendgroup=strat,
    ), row=1, col=1)
    fig.add_trace(go.Bar(
        x=[strat], y=[vol], name=strat,
        marker_color=color, text=[f"{vol:.1%}"], textposition="auto",
        showlegend=False, legendgroup=strat,
    ), row=1, col=2)
    fig.add_trace(go.Bar(
        x=[strat], y=[sr], name=strat,
        marker_color=color, text=[f"{sr:.2f}"], textposition="auto",
        showlegend=False, legendgroup=strat,
    ), row=1, col=3)

fig.update_yaxes(tickformat=".0%", row=1, col=1)
fig.update_yaxes(tickformat=".0%", row=1, col=2)
fig.update_layout(
    height=450, template="plotly_white",
    title_text="Strategy Performance Comparison",
    showlegend=False,
)
fig.show()

In [ ]:
# --- Cumulative return comparison ---
cum_max_sharpe = (1 + port_daily_returns).cumprod()
cum_min_var    = (1 + port_mv_returns).cumprod()
cum_equal_w    = (1 + port_equal_returns).cumprod()
cum_rp_w       = (1 + port_rp_returns).cumprod()

fig = go.Figure()

for cum, name, color in [
    (cum_max_sharpe, "Max Sharpe",   "indianred"),
    (cum_min_var,    "Min Variance", "steelblue"),
    (cum_equal_w,    "Equal Weight", "goldenrod"),
    (cum_rp_w,       "Risk Parity",  "seagreen"),
]:
    fig.add_trace(go.Scatter(
        x=cum.index, y=cum.values,
        mode="lines", name=name,
        line=dict(color=color, width=2),
    ))

fig.update_layout(
    title="Cumulative Return Comparison (All Strategies)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return (1 = start)",
    height=500,
    template="plotly_white",
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
)
fig.show()

In [ ]:
# --- Drawdown comparison ---
fig = go.Figure()

dd_max_sharpe = (cum_max_sharpe - cum_max_sharpe.cummax()) / cum_max_sharpe.cummax()

for dd, name, color in [
    (dd_max_sharpe, "Max Sharpe",   "indianred"),
    (dd_mv,         "Min Variance", "steelblue"),
    (dd_equal,      "Equal Weight", "goldenrod"),
    (dd_rp,         "Risk Parity",  "seagreen"),
]:
    fig.add_trace(go.Scatter(
        x=dd.index, y=dd.values,
        mode="lines", name=name,
        line=dict(color=color, width=1.5),
    ))

fig.update_layout(
    title="Drawdown Comparison (All Strategies)",
    xaxis_title="Date",
    yaxis_title="Drawdown",
    yaxis_tickformat=".0%",
    height=450,
    template="plotly_white",
    legend=dict(yanchor="bottom", y=0.01, xanchor="left", x=0.01),
)
fig.show()

---
## 8. Next Steps / Extension Points

This notebook provides a solid foundation for portfolio optimization research.
Below are concrete ideas for extending the analysis further.

### Black-Litterman Model
Combine market equilibrium returns with subjective investor views to produce
more stable expected return estimates. This reduces sensitivity to estimation
error compared to raw historical means.

### Hierarchical Risk Parity (HRP)
Use hierarchical clustering on the correlation matrix to build a tree-based
allocation. HRP avoids matrix inversion, making it more robust for
ill-conditioned covariance matrices. See Lopez de Prado (2016).

### Transaction Costs and Constraints
- Add proportional transaction costs to the objective function.
- Enforce sector exposure limits or minimum/maximum weight constraints.
- Test turnover constraints for realistic rebalancing.

### Real Market Data via yfinance
Replace synthetic data with real prices:
```python
import yfinance as yf
tickers = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
prices = yf.download(tickers, start="2022-01-01", end="2024-01-01")["Close"]
```

### Rebalancing Simulation
Simulate monthly or quarterly rebalancing with walk-forward optimization.
Track out-of-sample performance and compare strategies over rolling windows.

### Factor Models
- Decompose returns into Fama-French factors (market, size, value, momentum).
- Use factor-based covariance estimation for better out-of-sample behavior.
- Integrate with the project's `src.features.forensic` module for
  fundamental-factor overlays.

### Integration with Project Modules
When running locally (not on Colab), this notebook can import directly from
the project:
```python
from src.data.dummy_provider import DummyDataProvider
from src.features.portfolio.optimizer import MeanVarianceOptimizer
from src.core.types import PortfolioResult
```
The `OptimizerInterface` in `src.core.interfaces` provides a contract
for adding new optimization strategies (e.g., `RiskParityOptimizer`,
`BlackLittermanOptimizer`) that plug into the Streamlit app seamlessly.